In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import sys
import traceback
from rich.console import Console
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score
from sklearn.model_selection import train_test_split, cross_val_score  # Import cross_val_score
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from pydub import AudioSegment
import yarppg
import cv2 as cv
from deepface import DeepFace
from collections import Counter
from pydub import AudioSegment
import torch
from aniemore.recognizers.voice import VoiceRecognizer
from aniemore.recognizers.multimodal import VoiceTextRecognizer
from aniemore.recognizers.multimodal import MultiModalRecognizer
from aniemore.utils.speech2text import SmallSpeech2Text
from aniemore.models import HuggingFaceModel
import pickle

In [ ]:
console = Console()
emotion_counter = Counter()

# # Для распознавания интонации
# model = HuggingFaceModel.Voice.WavLM
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# vr = VoiceRecognizer(model=model, device=device)

# Мультимодальный распознаватор
model = HuggingFaceModel.MultiModal.WavLMBertFusion
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mr = MultiModalRecognizer(model=model, s2t_model=SmallSpeech2Text(), device=device)

# настраиваем распознаватор пульса
# fps = 30 #yarppg.get_video_fps(filename)
# filter_cfg = yarppg.digital_filter.FilterConfig(fps, 0.5, 1.5, btype="bandpass")
# livefilter = yarppg.digital_filter.make_digital_filter(filter_cfg)
# processor = yarppg.FilteredProcessor(yarppg.Processor(), livefilter=livefilter)
# rppg = yarppg.Rppg(
#     processor=processor,
#     hr_calc=yarppg.PeakBasedHrCalculator(fps, window_seconds=4),
# )

# сканируем папку эмоции и дергаем названия эмоций - вложенных папок
data_dir = 'emotions'
emotions = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]
console.print("Найдены эмоции:", emotions)

# смотрим скок файлов во вложенных папках
total = 0
for root, dirs, files in os.walk(data_dir):
    total += len(files)
console.print (total)

In [ ]:
# создаем прогрессбар
bar = tqdm(total= total)

# создаем списки для фич и соответствующих им эмоций
X = []
y = []

# залезаем по очереди в каждую папку с эмоцией
for emotion in emotions:
    emotion_dir = os.path.join(data_dir, emotion)# папка с эмоциями
    # получаем список файлов-видосов в ней
    files = [f for f in os.listdir(emotion_dir) if f.endswith('.mp4')]
    # и перебираем по очереди все файлы
    for file in files:
        # console.print(f"Обрабатываем файл: {file} Эмоция: {emotion}")
        # полный путь к файлу
        file_path = os.path.join(emotion_dir, file)
        # на всякий случай оборачиваем в трай-кэтч, вдруг что-то пойдет не так
        # чтоб программа не посыпалась
        try:
            ##############################################################################
            # начинаем с разпознавания эмоций по видео с помощью дипфейса
            # открываем видеофайл
            cap = cv.VideoCapture(file_path)

            # очищаем-обнуляем все списки-переменные
            emotion_counter.clear()
            statka = [] #очищаем массив
            new_statka = []
            final_statka = []
            cnt = 0 # счетчик кадров
            # это выпилить, можно просто массив засунуть
            v_angry = v_disgust = v_fear = v_happy = v_sad = v_surprise = v_neutral = []

            # пока файл открыт - работаем с кадрами
            while cap.isOpened():
                pass
                ret, frame = cap.read()
                # если вдруг чтение кадра выдало ошибку - брейкаем
                if not ret:
                    break

                # берем каждый 30-цатый кадр
                if cnt % 30 == 0:
                    # # находим на картинке лицо с помощью каскада хаара
                    # faceCascade= cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
                    # imgGray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)                              
                    # faces = faceCascade.detectMultiScale(imgGray,1.1,7)
                    # for (xx,yy,ww,hh) in faces:

                    #     # и оставляем только лицо, так дипфейс будет работать быстрее
                    #     frame = frame [yy:yy+hh, xx:xx+ww]

                    # анализируем лицо дипфейсом и получаем главную эмоцию кадра и проценты по всем эмоциям
                    result = DeepFace.analyze(frame, actions=['emotion'], 
                                            enforce_detection=False,  
                                            # detector_backend = 'retinaface',
                                            # detector_backend = 'mtcnn', 
                                            # detector_backend ='opencv',
                                            detector_backend = 'ssd'
                                            )

                    # сохраняем в список статка главную эмоцию кадра + проценты по эмоциям 
                    statka.append([result[0]["dominant_emotion"], result[0]["emotion"]])
                    # добавляем главную эмоцию кадра в высчитыватель главной эмоции видер
                    emotion_counter.update([result[0]["dominant_emotion"]])
                # счетчик кадра +1
                cnt += 1

                if cv.waitKey(1) & 0xFF == ord('q'):
                    break
            
            # после завершения обработки видоса, высчитываем главную эмоцию видоса
            # можно это не делать и взять название эмоции из папки
            # но т.к. датасет не идеален - в некоторых видео не обнаруживается эта эмоция :(
            most_common_emotion, _ = emotion_counter.most_common(1)[0]
            # конвертируем список в нумпай-список
            statka = np.array(statka)

            # оставляем только строки с доминантной эмоцией 
            # и в список нью_статка записываем только значения (вероятности) всех эмоций
            # их названия мы и так знаем, а позиции их статичны
            
            # тут можно заменить мост_коммон_эмошн на эмошн-название папки
            # но как я написал раньше - в некоторых видео вообще не совпадает главная эмоция с названием папки
            # и список нью статка получается пустым, а так хоть что-то

            # хотя можно попробовать сделать модель и без этих файлов и сравнить результаты
            new_statka = [list(x[1].values()) for x in statka if x[0] == most_common_emotion]
            # конвертируем список в нумпай-список для нумпай-операций
            new_statka = np.array(new_statka)

            # находим среднее арифметическое столбцов (аксис = 0)
            final_statka = np.mean(new_statka, axis=0)
            # превращаем экспон. значения в обычные дроби
            final_statka = [float(x) for x in final_statka]

            v_angry     = [final_statka[0]]
            v_disgust   = [final_statka[1]]
            v_fear      = [final_statka[2]]
            v_happy     = [final_statka[3]]
            v_sad       = [final_statka[4]]
            v_surprise  = [final_statka[5]]
            v_neutral   = [final_statka[6]]

            # освобождаем-очищаем
            cap.release()
            # cv.destroyAllWindows()

            ###############################################################################
            # анализируем аудио и текст с помощью аниморе

            audio_features_vals = []
            a_angry = a_disgust = a_fear = a_happy = a_sad = a_surprise = a_neutral = []

            # дергаем звук из видео и сохраняем его в темп.вав
            mp4_version = AudioSegment.from_file(file_path, "mp4")
            mp4_version.export("temp.wav", format="wav")

            # запускаем распознаватор эмоций по аудио
            # audio_features = vr.recognize("temp.wav", return_single_label=False)

            # запускаем мультимодальный распознаватор эмоций
            audio_features = mr.recognize("temp.wav", return_single_label=False)

            # получаем статистику - сколько % каждой эмоции в аудио
            audio_features_vals = list(audio_features.values()) 

            a_angry     = [audio_features_vals[0] * 100]
            a_disgust   = [audio_features_vals[1] * 100]
            a_fear      = [audio_features_vals[3] * 100]
            a_happy     = [audio_features_vals[4] * 100]
            a_sad       = [audio_features_vals[6] * 100]
            a_surprise  = [audio_features_vals[2] * 100]
            a_neutral   = [audio_features_vals[5] * 100]

            #####################################################################################
            
            # # делаем фиче-вектор, складываем все списки с результатами в один большой результат
            features = np.concatenate((
                # сюда запихнуть все данные
                v_angry, v_disgust, v_fear, v_happy, v_sad, v_surprise, v_neutral
                # потом запихнуть аудио
                , a_angry, a_disgust, a_fear, a_happy, a_sad, a_surprise, a_neutral
                # и пульс
            ))

            # console.print (features)

            # # Запихиваем в списки, в х - вектор, в у - название эмоции для пос-го обучения
            X.append(features)  
            y.append(emotion) 

        # ну мало ли, вдруг что-то отвалится
        except AttributeError:
            pass
            traceback.print_exc()
        except Exception as e:
            # AttributeError
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            # print(f"Error loading {file}: {e.with_traceback}")   

        bar.update(1) # увеличиваем на единицу
bar.close() # Заканчиваем работу бара

# конвертируем в нампай-списки
X = np.array(X)
y = np.array(y)

console.print(f"Число фиче-векторов: {len(X)}")
console.print(f"Число эмоций: {len(y)}") 

In [ ]:
# тут начинается обучение

# делим списки на трейн и тест 80 к 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация фич с помощью StandardScaler
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)  # Fit to the training data and transform
X_test = scaler.transform(X_test)  # Only transform the test data

# Кодирование эмоций
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # конвертируем названия эмоций в числовые значения
y_test_encoded = label_encoder.transform(y_test)

Ниже попробуем 4 классификатора, посмотрим, какой из них точнее

In [ ]:
# метод опорных векторов
from sklearn.svm import SVC
model = SVC(decision_function_shape="ovo")

In [ ]:
# случайный лес
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)

In [ ]:
# многослойный персептрон
from sklearn.neural_network import MLPClassifier
model_params = {
        "alpha": 0.01,
        "batch_size": 32,
        "epsilon": 1e-08,
        "hidden_layer_sizes": (300,),
        "learning_rate": "adaptive",
        "max_iter": 500
    }
   
# model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)
model = MLPClassifier(**model_params)

In [ ]:
# деревья с градиентным бустингом
# будем использовать этот классификатор как наиболее быстрый и точный
# model = XGBClassifier(eval_metric='mlogloss',objective="binary:logistic")
model = XGBClassifier(n_estimators=1000)

In [ ]:
# пробуем сделать 5-шаговую кросс-валидацию
cv_scores = cross_val_score(model, X_train, y_train_encoded, cv=5)
console.print(f'Cross-Validation Scores: {cv_scores}')
console.print(f'Mean Cross-Validation Score: {np.mean(cv_scores):.2f}')

# обучаем модель
model.fit(X_train, y_train_encoded)

In [ ]:
# получаем предсказания на тестовом наборе
y_pred = model.predict(X_test)

# получаем названия эмоций, декодируем у-результаты
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test_encoded)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
# Высчитываем точность
accuracy = accuracy_score(y_test_encoded, y_pred)
console.print(f'Accuracy: {accuracy:.2f}')

precision = precision_score(y_test_encoded, y_pred, average='weighted')
console.print(f'Precision: {precision:.2f}')

recall = recall_score(y_test_encoded, y_pred, average='weighted')
console.print(f'Recall: {recall:.2f}')

f1 = f1_score(y_test_encoded, y_pred, average='weighted')
console.print(f'F1: {f1:.2f}')

# выводим отчет о классификации
console.print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

# показываем на экране Confusion Matrix
conf_matrix = confusion_matrix(y_test_encoded, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.ylabel('Реальные')
plt.xlabel('Предсказанные')
plt.title('Confusion Matrix: Actual vs Predicted')
plt.show()

# # показываем на экране Plot Feature Importance
# plt.figure(figsize=(12, 10))
# plot_importance(model, max_num_features=10, importance_type='weight')
# plt.title('Feature Importance')
# plt.show()

In [ ]:
# сохраняем модель в файл
filename = 'my_model.sav'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# загрузка модели из файла
# with open(filename, 'rb') as file:
#     pickle_model = pickle.load(file)

# # Calculate the accuracy score and predict target values
# score = pickle_model.score(X_test, y_test_encoded)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model.predict(X_test)